# Plotting the HR Diagram Interactive

In [1]:
# This Jupyter Notebook was initially developed by Juan Cabanela over the Summer of 2019
# to allow students to more easily created HR diagrams which require plotting on logarithmic
# axes, which is well beyond what most intro students feel comfortable with.

from IPython.display import display
import numpy as np
import bqplot as bq
from traitlets import link
import ipywidgets as widgets
import tempNcolor as t2c
import ipysheet as ipysheet

In [2]:
# Initial Stellar Data
star_names = ["18 Scorpii", "Sirius", "EGGR 102", "mu Geminorum", 
              "NSV 14168", "NLTT 40117", "zeta Andromedae", 
              "HD 84183", "Alkaid", "Altair", "Fomalhaut", "Pollux", 
              "Van Maanen\'s Star", "NLTT 42064", "Gliese 269-17"]
star_names_array = np.array(star_names)
star_num = len(star_names)

star_BV = np.array([0.65, 0.01, -0.09, 1.62, 1.65, 1.10, 1.10, 0.60, 
                    -0.10, 0.22, 0.09, 1.00, 0.56, 0.86, 1.44])
star_brightness= np.array([0.000429, 0.263, 5.29E-7, 0.00464, 9.92E-6, 
                           5.16E-6, 0.00156, 0.000130, 0.0124, 0.0335, 
                           0.0234, 0.0236, 7.74E-7, 9.65E-5, 3.91E-6])
star_parallax = np.array([0.071, 0.38, 0.040, 0.014, 0.25, 0.025, 0.018, 
                          0.013, 0.032, 0.086, 0.051, 0.092, 0.064, 
                          0.064, 0.053])

# Compute RGB colors for points based on B-V color
hexcolors = t2c.rgb2hex(t2c.bv2rgb(star_BV))
hexlist = hexcolors.tolist()

# Compute the answers (to allow setting of plot ranges)
star_dist = 1/star_parallax
star_lum = star_brightness*(4*np.pi*star_dist**2)

# Decide what goes in the columns and plot for distance and luminosity
dist4plot = np.zeros_like(star_dist)
lum4plot  = 1e-5*np.ones_like(star_lum)
opacities4plot = np.ones_like(star_lum)
names4plot = np.copy(star_names_array)
# names4plot[:] = ' '

In [3]:
##
## Define scales, axes, and tooltips, which could be similar between 
## various implementations of the displayed widgets
##

# Scales to transform the data
min_loglum = np.floor(np.min(np.log10(star_lum)))
max_loglum = np.ceil(np.max(np.log10(star_lum)))
min_lum = 10**min_loglum
max_lum = 10**max_loglum
n_lum = np.int(max_loglum-min_loglum+1)
lum_ticks = 10**np.linspace(min_loglum, max_loglum, n_lum)

min_logbright = np.floor(np.min(np.log10(star_brightness)))
max_logbright = np.ceil(np.max(np.log10(star_brightness)))
min_bright = 10**min_logbright
max_bright = 10**max_logbright
n_bright = np.int(max_logbright-min_logbright+1)

x_sc_color = bq.LinearScale(min =-1, max=2.5)
y_sc_bright = bq.LogScale(min = min_bright, max=max_bright)
y_sc_lum = bq.LogScale(min = min_lum, max=max_lum)

# Labels and scales for Axes
ax_x_color = bq.Axis(label='B-V', scale=x_sc_color)
ax_y_bright = bq.Axis(label='Brightness (L_sun/pc^2)', scale=y_sc_bright, orientation='vertical')
ax_y_lum = bq.Axis(label='Luminosity (L_sun)', scale=y_sc_lum, orientation='vertical', tick_values=lum_ticks)

# moving the label perpendicular to the axis
ax_y_bright.label_offset = '3.5em'
ax_y_lum.label_offset = '3em'


In [4]:
# Build an interactive data sheet
sheet = ipysheet.sheet(rows=star_num, columns=6, column_headers=["Star Name", "B-V", "Brightness", "Parallax", "Distance", "Luminosity"])
names = ipysheet.column(0, names4plot, row_start=0, background_color="lightgray")
BVs = ipysheet.column(1, star_BV, row_start=0, numeric_format='0.00', type='numeric')
fluxes = ipysheet.column(2, star_brightness, row_start=0, numeric_format='0.000000000', type='numeric')
parallaxes = ipysheet.column(3, star_parallax, row_start=0, type='numeric')
distances = ipysheet.column(4, dist4plot, row_start=0, type='numeric')
lums = ipysheet.column(5, lum4plot, row_start=0, type='numeric')


In [5]:
# This function takes the information from the BrushSelector and acts on it
def update_plot(change):
    global opacities4plot, names4plot
    
    # Change the parameters
    print("Making change: ", lums.value)
    #scat_lum.y = change.value
    #opacities4plot[change.value > 0] = 1
    #names4plot[change.value > 0] = star_names[change.value > 0]
    #scat_lum.default_opacities = opacities4plot
    #scat_lum.names = names4plot

def on_value_change(change):
    caption.value = change['new']

In [8]:
##
## Construct the actual plots and widgets for the first interactive
##
PlotBkgStyle={'fill': 'lightgrey'}
PlotLayout={'width': '400px', 'min_height': '400px'}

# Define a tooltip
#def_tt_bright = bq.Tooltip(fields=['name', x', 'y'], labels=['Name', B-V', 'Brightness'])
def_tt_lum = bq.Tooltip(fields=['name', 'x', 'y'], labels=['Name', 'B-V', 'Luminosity'])


# Designing the Brightness diagram
#scat_bright = bq.Scatter(x=star_BV, y=star_brightness, 
#                         names=names4plot, display_names=True, apply_clip=False,
#                         scales={'x': x_sc_color, 'y': y_sc_bright},
#                         marker='circle', colors=hexlist, 
#                         default_size=30, tooltip=def_tt_bright, stroke='black', fill=True)
#fig_bright = bq.Figure(axes=[ax_x_color, ax_y_bright], marks=[scat_bright], 
#                       title='Brightness of Sample Stars', 
#                       background_style=PlotBkgStyle, layout=PlotLayout)

# Designing the HR diagram (set points to be initiallize invisible)
scat_lum = bq.Scatter(x=star_BV, y=lum4plot, 
                      names=names4plot, display_names=True, apply_clip=True,
                      scales={'x': x_sc_color, 'y': y_sc_lum},
                      marker='circle', colors=hexlist, default_opacities=list(opacities4plot),
                      default_size=30, tooltip=def_tt_lum, stroke='black', fill=True)
scat_lum.tooltip = None
fig_lum = bq.Figure(axes=[ax_x_color, ax_y_lum], marks=[scat_lum], 
                    title='HR Diagram of Sample Stars', 
                    background_style=PlotBkgStyle, layout=PlotLayout)


# Displaying the data

# Link diagram information to sheet information
#fig_bright.layout.height = '400px'
#fig_bright.layout.width = '400px'
#sheet.layout.height = '400px'
#sheet.layout.width = '600px'
#link((scat_bright, 'names'), (names, 'value'))
#link((scat_bright, 'x'), (BVs, 'value'))
#link((scat_bright, 'y'), (fluxes, 'value'))

# Display the HR diagram and the sheet
fig_lum.layout.height = '400px'
fig_lum.layout.width = '400px'
sheet.layout.height = '400px'
sheet.layout.width = '600px'
link((scat_lum, 'names'), (names, 'value'))
link((scat_lum, 'x'), (BVs, 'value'))
link((scat_lum, 'y'), (lums, 'value'))

# Respond to changes in the luminosity values
#lums.observe(on_value_change, names='value')

# Display the HR diagram and the sheet
caption = widgets.Label(value='Caption to Update')
HR_widget = widgets.HBox([sheet, fig_lum])
display(HR_widget)